### A note on getters and setter

#### Lets redefine the Animal class

In [12]:
from enum import Enum
from IPython.display import Image
from abc import abstractmethod


# Let's define some Enums first to represent Cat Colors
class CatColor(Enum):
    BLACK = "blaack"
    WHITE = "white"
    GRAY = "grey"
    ORANGE = "orange"
    BROWN = "Brown"
    SPOTTED = "spotted"
    TABBY = "tabby"


class Species(Enum):
    CAT = "cat"
    BIRD = "bird"
    MOUSE = "mouse"
    DOG = "dog"


In [13]:
class Animal:
    def __init__(self, name: str, species: Species, is_female: bool):
        """Lets define a constructor for the Animal class
        This can be used to instatiate an object of the class...

        IMPORTANT: The setters and getter are not defined here, but they could be...
        See later notes on getters and setters...
        The class definitions are intentionally left simple for clarity...

        NOTES: I wanted to avoid using the word 'sex' in a corporate environment
        this is why the true/false flag is_female was introduced.

        Args:
            name (str): name of the animal
            species (Species): the species
            is_female (bool): True if female, False if male
        """
        self._name = name
        self._species = species
        self._is_female = is_female
        print(f"Animal constructor call: {self._name}")

    # Getter and setter for name
    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value: str):
        if not value:
            raise ValueError("Name cannot be empty.")
        self._name = value

    # Getter and setter for species
    @property
    def species(self):
        return self._species

    @species.setter
    def species(self, value: str):
        if not value:
            raise ValueError("Species cannot be empty.")
        self._species = value

    # Getter and setter for sex
    @property
    def male_female(self) -> str:
        """A not so nicely named method.
        Defaults to "male" if the _is_female flag is None

        Returns:
            _type_: string
        """
        return "female" if self._is_female else "male"

    @male_female.setter
    def male_female(self, value: bool):
        if not value:
            raise ValueError("The flag cannot be empty.")
        self._is_female = value

    def __str__(self):
        """magic method or dunder method"""
        return f"Name: {self.name} & Type: {self.species}"

    def makes_a_sound(self):
        print(f"{self.name} is making a sound typical of its species...")

Using property-based getters and setters has some advantage on the original solution:

  - Encapsulation & Data Protection: Properties allow you to control how attributes are accessed and modified. Validation can be added...

  - Future changes: 
The code may need changes later, using this constructs the interior of the class logic can be easily and freely modified. Breaking existing code is not an issue here, the class interface can be also kept unchanged.



  - Readability & Consistency: The syntax is kept simple (animal.name instead of animal.get_name()), but still gets the benefits of method calls.

  - Debugging & Logging: Logging can be added to these methods, to track where the values are accessed or changed.

  - Compatibility with Existing Code: Properties allow the dev to start with public attributes and later add logic without changing how the attribute is accessed from outside the class.

### Class level attributes / methods?

In [21]:
class Animal:
    def __init__(self, name):
        self._name = name
        self._species = "Generic Animal"


class Cat(Animal):
    _species = "Cat"


cat = Cat("Lujzi")
print(
    cat._species
)  # Output: "Generic Animal" (instance-level wins even if its in the parent)
print(
    Cat._species
)  # Output: "Cat" (class-level attribute is presented if called on class level)
# What happened here?
# Did we mess up attribute resolution?

Generic Animal
Cat


In [22]:
## an alternative design choice for the Animal/Cat class


class Animal:
    species = "Generic Animal"  # Class attribute
    # We no longer allow new animal to be created with different species values
    # this is a shared attribute for all instances of Animal and its subclasses

    def __init__(self, name):
        self._name = name  # Instance attribute

    @classmethod  # yes, this is a class method
    def change_species(cls, new_species):
        """
        This method allows controlled manipulation of the class-level attribute.
        The cls is used instead of self to highlight that this method operates on the class itself
        """
        cls.species = new_species

In [23]:
# Instantiate two animals
mouse = Animal("Mickey")
birdy = Animal("Birdy")

# Show initial species
print(f"\nInitial species for {mouse._name}: {mouse.species}")
print(f"Initial species for {birdy._name}: {birdy.species}")

# Change species using class method
Animal.change_species("Mouse")

# Show updated species
print(f"\nAfter changing species:")
print(f"Species for {mouse._name}: {mouse.species}")
print(f"Species for {birdy._name}: {birdy.species}")


Initial species for Mickey: Generic Animal
Initial species for Birdy: Generic Animal

After changing species:
Species for Mickey: Mouse
Species for Birdy: Mouse


### Static methods

In [17]:
# Static method example to explain utility functions


class Animal:
    species = "Generic Animal"  # Class attribute
    # We no longer allow new animal to be created with different species values
    # this is a shared attribute for all instances of Animal and its subclasses

    def __init__(self, name):
        self._name = name  # Instance attribute

    @staticmethod
    def is_valid_name(name: str) -> bool:
        return name.isalpha() and len(name) > 2

A static method have the following features:
- Belongs to the class (not the instance).
- Does not take self or cls as its first argument.
- Is used for functionality that’s related to the class, but doesn’t access or modify class or instance level attributes.


<br> Note that the above example take the "name" input argurment, but does not look into the classes internal state


In [18]:
print(Animal.is_valid_name("Mici"))  # True
print(Animal.is_valid_name("123"))  # False

True
False


#### Dataclasses?

In [20]:
from dataclasses import dataclass


@dataclass(frozen=True)
class User:
    nev: str
    phone_number: str

u1 = User("Laci", "32432")
u2 = User("Laci", "gfff")

print(f"u1 == u2 : {u1==u2}")
print(f"u1 != u2 : {u1!=u2}")

print(
    f"u1 : {hash(u1)}"
)  # available if the dataclass is immutable (frozen=True)
print(str(u1))

u1 == u2 : False
u1 != u2 : True
u1 : 1714406706924647798
User(nev='Laci', phone_number='32432')


#### The Owner class could have been a dataclass...

Advantages:
  - Auto-generates common methods -> Less boilerplate code (no custom __str__ or __eq__)
  - Type hints are encouraged and enforced
  - Customizable behavior -> the common methods can still be overriden
  - Immutability support -> see the frozen=True